In [1]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import linear_model
import gzip
from collections import defaultdict

In [7]:
f = open("RAW_interactions.csv",encoding="utf8")
dataset = []
for l in f:
    dataset.append(l)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 658: character maps to <undefined>